In [ ]:
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
import tensorflow as tf
import numpy as np
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'{\r\n        "intents": [\r\n            {\r\n                "tag": "greeting",\r\n                "patterns": [\r\n                    "Hi",\r\n                    "How are you",\r\n                    "Is anyone there?",\r\n                    "Hello",\r\n                    "Good day"\r\n                ],\r\n                "responses": [\r\n                    "Hello, thanks for visiting",\r\n                    "Good to see you again",\r\n                    "Hi there, how can I help?"\r\n                ]\r\n            },\r\n            {\r\n                "tag": "goodbye",\r\n                "patterns": [\r\n                    "Bye",\r\n                    "See you later",\r\n                    "Goodbye"\r\n                ],\r\n                "responses": [\r\n                    "See you later, thanks for visiting",\r\n                    "Have a nice day",\r\n                    "Bye! Come back again soon."\r\n                ]\r\n            },\r\n 

In [ ]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [ ]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'chatbot',
   'patterns': ['Who built this chatbot?',
    'Tell me about Chatbot',
    'Who developed this Chatbot?'],
   'responses': ['Hi, I am a Chatbot designed by Ankit.',
    'Thanks for asking. I am designed by Ankit Kumar.',
    'I am a chatbot. I was designed by Ankit.']},
  {'tag': 'name',
   'patterns': ['What is the name of this Chatbot?',
    'Tell me your name?',
    'Do you have a name?'],
   'responses': ['Hi, I am Tu

In [ ]:
words = []
classes = []
documents = []
ignore = ['?']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

39 documents
12 classes ['about', 'chatbot', 'connect', 'food', 'goodbye', 'greeting', 'jokes', 'location', 'movies', 'music', 'name', 'thanks']
70 unique stemmed words ["'m", "'s", 'a', 'about', 'account', 'address', 'ani', 'anyon', 'are', 'built', 'bye', 'can', 'chatbot', 'connect', 'day', 'develop', 'dinner', 'do', 'favorit', 'food', 'for', 'give', 'good', 'goodby', 'have', 'hello', 'help', 'hi', 'how', 'hungri', 'i', 'is', 'joke', 'know', 'later', 'laugh', 'link', 'locat', 'make', 'me', 'media', 'movi', 'music', 'name', 'of', 'out', 'play', 'reach', 'recommend', 'see', 'social', 'some', 'song', 'suggest', 'tell', 'thank', 'that', 'the', 'there', 'thi', 'to', 'way', 'we', 'what', 'where', 'which', 'who', 'you', 'your', 'yourself']


In [ ]:
training = []
output = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

<ipython-input-7-f4723073c0fd>:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10,input_shape=(len(train_x[0]),)))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax'))
model.compile(tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=8, verbose=1)
model.save("model.pkl")

Epoch 1/100
5/5 [==============================] - 1s 5ms/step - loss: 2.4540 - accuracy: 0.1538
Epoch 2/100
5/5 [==============================] - 0s 3ms/step - loss: 2.4302 - accuracy: 0.1538
Epoch 3/100
5/5 [==============================] - 0s 4ms/step - loss: 2.4089 - accuracy: 0.1538
Epoch 4/100
5/5 [==============================] - 0s 3ms/step - loss: 2.3890 - accuracy: 0.1538
Epoch 5/100
5/5 [==============================] - 0s 3ms/step - loss: 2.3677 - accuracy: 0.1538
Epoch 6/100
5/5 [==============================] - 0s 3ms/step - loss: 2.3478 - accuracy: 0.2051
Epoch 7/100
5/5 [==============================] - 0s 3ms/step - loss: 2.3271 - accuracy: 0.2051
Epoch 8/100
5/5 [==============================] - 0s 3ms/step - loss: 2.3073 - accuracy: 0.2051
Epoch 9/100
5/5 [==============================] - 0s 3ms/step - loss: 2.2884 - accuracy: 0.2051
Epoch 10/100
5/5 [==============================] - 0s 3ms/step - loss: 2.2668 - accuracy: 0.2308
Epoch 11/100
5/5 [===========

In [ ]:
import pickle
pickle.dump( {'words':words, 'classes':classes}, open( "training_data", "wb" ) )

In [ ]:
from keras.models import load_model
model = load_model("model.pkl")

In [ ]:
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']

In [ ]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [ ]:
def clean_up_sentence(sentence):

    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
    bag=np.array(bag)
    return(bag)

In [ ]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    bag = bow(sentence, words)
    results = model.predict(np.array([bag]))
    results = [[i,r] for i,r in enumerate(results[0]) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

def response(sentence):
    results = classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    return print(random.choice(i['responses']))

            results.pop(0)

In [ ]:
response('How are you?')

1/1 [==============================] - 0s 93ms/step
Good to see you again


In [ ]:
response('Suggest me a movie')

1/1 [==============================] - 0s 24ms/step
Shawshank Redemption is one of the best movies ever!


In [ ]:
response('Tell me a joke')

1/1 [==============================] - 0s 21ms/step
I'm reading a book on anti-gravity. It's impossible to put down!


In [ ]:
response('Who developed this Chatbot?')

1/1 [==============================] - 0s 32ms/step
Thanks for asking. I am designed by Ankit Kumar.


In [ ]:
response('Do you have a name?')

1/1 [==============================] - 0s 20ms/step
You can call me Turing. I was named after the famous scientist who remained unknown for 50 years.


In [ ]:
response('Okay. Bye')

1/1 [==============================] - 0s 21ms/step
Have a nice day
